In [136]:
from ortools.linear_solver import pywraplp
from numpy import average, std
import json
import pandas as pd
import requests
import math
import numpy as np

In [137]:
response1 = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones_barrios.json")
distances_stations_ntas_db = json.loads(response1.text)

In [138]:
total_vehicles = {
    "ha": 1,
    "engines": 197,
    "ladders": 143,
    "rescue": 5,
    "squads": 8,

}

In [139]:
with open("./functions/neighbors_with_9_minute_station.json", "r") as f:
    neighbors_with_9_minute_station = json.load(f)
neighbourhoods = list(neighbors_with_9_minute_station.keys())
K = len(neighbourhoods)
shifts = ["9 AM to 6 PM", "6 PM until 9 AM"]
I = len(shifts)
stations = list(distances_stations_ntas_db.keys())
L = len(stations)
vehicles = ["engines", "squads", "ladders", "ha", "rescue"]
J = len(vehicles)

In [140]:
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/estaciones.json")
stations_db = json.loads(response.text)
Cl = {}
for st in stations_db:
    Cl[stations.index(st["address"])] = st["capacity"]

In [141]:
def index2shift(i):
    return shifts[i]

def shift2index(shift):
    return shifts.index(shift)

def index2vehicle(j):
    return vehicles[j]

def vehicle2index(vehicle):
    return vehicles.index(vehicle)

def index2neighbourhood(k):
    return neighbourhoods[k]

def neighbourhood2index(neighbourhood):
    return neighbourhoods.index(neighbourhood)

def index2station(l):
    return stations[l]

def station2index(station):
    return stations.index(station)
    

Compute the costs for vehicles of type ___hazmat___, ___rescue___, and ___squad___ for assigning them to an specific station in a determined shift

In [142]:
b = pd.read_csv("./aux_data/bijk.csv",index_col=0) #Load estimated demands per vehicle and shift
b

,nta,engines,squads,ladders,ha,rescue,shift
0,BK17,4.934514,0.188820,3.806345,0.028029,0.113781,1
1,BK25,3.671613,0.159006,2.610501,0.018974,0.099957,1
2,BK27,2.396166,0.125880,1.681814,0.011643,0.062739,1
3,BK28,5.043240,0.208696,3.684216,0.028029,0.129732,1
4,BK30,3.378887,0.162319,2.236482,0.016818,0.086134,1
...,...,...,...,...,...,...,...
84,QN52,1.802352,0.023188,1.094070,0.009487,0.005317,0
85,QN55,4.871787,0.039752,2.948900,0.025011,0.057422,0
86,QN62,1.250356,0.013251,0.844724,0.009918,0.011697,0
87,SI07,4.290518,0.046377,2.554526,0.038379,0.034028,0


In [143]:
# Load reacheable matrix (binary), indicating if a station (row) reaches a neighbourhood center (column) in
# less than 9 minutes
s = pd.read_csv("./aux_data/skl.csv",index_col=0)
s.index = pd.Series(list(s.index)).apply(lambda x: station2index(x))
s.columns = pd.Series(list(s.columns)).apply(lambda x: neighbourhood2index(x))

In [144]:
# Load distance matrix from station (row) to neighbourhood center (column)
d = pd.read_csv("./aux_data/dlk.csv",index_col=0)
d.index = pd.Series(list(d.index)).apply(lambda x: station2index(x))
d.columns = pd.Series(list(d.columns)).apply(lambda x: neighbourhood2index(x))

If we apply the product of reacheable matrices and distance matrix we get a matrix where adding the distances per row will serve as a cost for setting certain types of vehicles in that station.

This happens because our model indicates that when setting a hazmat, rescue or squat vehicle in a station, it will serve all the neighbourhoods which center is reacheable in less than 9 minutes

In [145]:
# The cost of assigning a vehicle (hazmat, rescue or squad) to a station in a certain shift will result from 
#the weighted combination of the distances of the reachable neighbourhoods depending on the estimated demand
cost_matrix = d*s
cost = {}
for i in range(len(b)):
      cost[(neighbourhood2index(b.iloc[i]["nta"]), "ha", b.iloc[i]["shift"])] = b.iloc[i]["ha"]
      cost[(neighbourhood2index(b.iloc[i]["nta"]), "squads", b.iloc[i]["shift"])] = b.iloc[i]["squads"]
      cost[(neighbourhood2index(b.iloc[i]["nta"]), "rescue", b.iloc[i]["shift"])] = b.iloc[i]["rescue"]

costs = {}
for i in range(len(cost_matrix)):
      costs[("ha", 0)] = costs.get(("ha", 0), []) + [sum(cost_matrix.iloc[i][j]*((0.1-cost.get((j, "ha", 0), 0))/0.1) for j in range(cost_matrix.shape[1]))]
      costs[("ha", 1)] = costs.get(("ha", 1), []) + [sum(cost_matrix.iloc[i][j]*((0.1-cost.get((j, "ha", 1), 0))/0.1) for j in range(cost_matrix.shape[1]))]
      costs[("squads", 0)] = costs.get(("squads", 0), []) + [sum(cost_matrix.iloc[i][j]*((0.56-cost.get((j, "squads", 0), 0))/0.56) for j in range(cost_matrix.shape[1]))]
      costs[("squads", 1)] = costs.get(("squads", 1), []) + [sum(cost_matrix.iloc[i][j]*((0.56-cost.get((j, "squads", 1), 0))/0.56) for j in range(cost_matrix.shape[1]))]
      costs[("rescue", 0)] = costs.get(("rescue", 0), []) + [sum(cost_matrix.iloc[i][j]*((0.37-cost.get((j, "rescue", 0), 0))/0.37) for j in range(cost_matrix.shape[1]))]
      costs[("rescue", 1)] = costs.get(("rescue", 1), []) + [sum(cost_matrix.iloc[i][j]*((0.37-cost.get((j, "rescue", 1), 0))/0.37) for j in range(cost_matrix.shape[1]))]

cost_matrix["costhashift0"] = costs[("ha", 0)]
cost_matrix["costhashift1"] = costs[("ha", 1)]
cost_matrix["costsquadsshift0"] = costs[("squads", 0)]
cost_matrix["costsquadsshift1"] = costs[("squads", 1)]
cost_matrix["costrescueshift0"] = costs[("rescue", 0)]
cost_matrix["costrescueshift1"] = costs[("rescue", 1)]
cost_matrix["reacheable_neighbourhoods"] = s.apply(lambda x:sum(x),axis=1)  #total reacheable centers
cost_matrix["normalized_costhashift0"] = cost_matrix.apply(lambda x: x["costhashift0"]/x["reacheable_neighbourhoods"],axis=1)
cost_matrix["normalized_costhashift1"] = cost_matrix.apply(lambda x: x["costhashift1"]/x["reacheable_neighbourhoods"],axis=1)
cost_matrix["normalized_costsquadsshift0"] = cost_matrix.apply(lambda x: x["costsquadsshift0"]/x["reacheable_neighbourhoods"],axis=1)
cost_matrix["normalized_costsquadsshift1"] = cost_matrix.apply(lambda x: x["costsquadsshift1"]/x["reacheable_neighbourhoods"],axis=1)
cost_matrix["normalized_costrescueshift0"] = cost_matrix.apply(lambda x: x["costrescueshift0"]/x["reacheable_neighbourhoods"],axis=1)
cost_matrix["normalized_costrescueshift1"] = cost_matrix.apply(lambda x: x["costrescueshift1"]/x["reacheable_neighbourhoods"],axis=1)
cost_matrix

C:\Users\Hugo\AppData\Local\Temp\ipykernel_15856\869074431.py:26: RuntimeWarning: invalid value encountered in double_scalars
  cost_matrix["normalized_costhashift0"] = cost_matrix.apply(lambda x: x["costhashift0"]/x["reacheable_neighbourhoods"],axis=1)
C:\Users\Hugo\AppData\Local\Temp\ipykernel_15856\869074431.py:27: RuntimeWarning: invalid value encountered in double_scalars
  cost_matrix["normalized_costhashift1"] = cost_matrix.apply(lambda x: x["costhashift1"]/x["reacheable_neighbourhoods"],axis=1)
C:\Users\Hugo\AppData\Local\Temp\ipykernel_15856\869074431.py:28: RuntimeWarning: invalid value encountered in double_scalars
  cost_matrix["normalized_costsquadsshift0"] = cost_matrix.apply(lambda x: x["costsquadsshift0"]/x["reacheable_neighbourhoods"],axis=1)
C:\Users\Hugo\AppData\Local\Temp\ipykernel_15856\869074431.py:29: RuntimeWarning: invalid value encountered in double_scalars
  cost_matrix["normalized_costsquadsshift1"] = cost_matrix.apply(lambda x: x["costsquadsshift1"]/x["reac

,0,1,2,3,4,5,6,7,8,9,...,costsquadsshift1,costrescueshift0,costrescueshift1,reacheable_neighbourhoods,normalized_costhashift0,normalized_costhashift1,normalized_costsquadsshift0,normalized_costsquadsshift1,normalized_costrescueshift0,normalized_costrescueshift1
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2404.720000,2359.216419,2404.720000,6.0,378.627748,400.786667,394.464849,400.786667,393.202737,400.786667
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2955.681925,3159.040989,2945.902589,8.0,358.227103,380.166797,394.452011,369.460241,394.880124,368.237824
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3119.655788,3404.216190,3115.459250,9.0,345.453494,359.916697,379.906794,346.628421,378.246243,346.162139
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2557.658530,2807.072689,2554.329693,8.0,316.127690,333.003240,352.678826,319.707316,350.884086,319.291212
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3748.172949,4016.279745,3742.183415,11.0,334.295584,351.491135,366.308745,340.742995,365.116340,340.198492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1290.720000,1290.720000,1290.720000,4.0,322.680000,322.680000,322.680000,322.680000,322.680000,322.680000
215,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,655.680000,655.680000,655.680000,3.0,218.560000,218.560000,218.560000,218.560000,218.560000,218.560000
216,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,872.470000,872.470000,872.470000,2.0,436.235000,436.235000,436.235000,436.235000,436.235000,436.235000
217,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1428.920000,1428.920000,1428.920000,4.0,357.230000,357.230000,357.230000,357.230000,357.230000,357.230000


In [146]:
factor_dict = {}
for v in ["ha","squads","rescue"]:
    for i in [0,1]:
        for n in list(b["nta"].unique()):
            factor_dict[(i,neighbourhood2index(n),v)] = b[v].loc[(b["nta"]==n)&(b["shift"]==i)].values
            
factor_dict

{(0, 79, 'ha'): array([0.01078051]),
 (0, 109, 'ha'): array([], dtype=float64),
 (0, 59, 'ha'): array([0.00862441]),
 (0, 110, 'ha'): array([], dtype=float64),
 (0, 94, 'ha'): array([], dtype=float64),
 (0, 95, 'ha'): array([], dtype=float64),
 (0, 90, 'ha'): array([], dtype=float64),
 (0, 53, 'ha'): array([0.03277275]),
 (0, 100, 'ha'): array([], dtype=float64),
 (0, 101, 'ha'): array([], dtype=float64),
 (0, 74, 'ha'): array([0.02673566]),
 (0, 91, 'ha'): array([], dtype=float64),
 (0, 92, 'ha'): array([], dtype=float64),
 (0, 68, 'ha'): array([0.04312204]),
 (0, 76, 'ha'): array([0.01034929]),
 (0, 75, 'ha'): array([0.02156102]),
 (0, 129, 'ha'): array([], dtype=float64),
 (0, 125, 'ha'): array([], dtype=float64),
 (0, 126, 'ha'): array([], dtype=float64),
 (0, 124, 'ha'): array([], dtype=float64),
 (0, 116, 'ha'): array([], dtype=float64),
 (0, 180, 'ha'): array([0.01768003]),
 (0, 147, 'ha'): array([0.05131522]),
 (0, 177, 'ha'): array([0.0297542]),
 (0, 178, 'ha'): array([0.02069

Define a function for simpler access to costs

In [147]:
def get_costs(l, vehicle, shift):
    return cost_matrix["normalized_cost" + vehicle + f"shift{shift}"][l]

In [148]:
Bijk = pd.read_csv("./aux_data/bijk.csv", index_col=0)
Bijk.columns = ["nta"] + vehicles + ["shift"]
newcols = []
for col in Bijk.columns[1:-1]:
    newcols.append(vehicle2index(col))
Bijk.columns = ["nta"] + newcols + ["shift"]
Bijk["nta"] = Bijk["nta"].apply(lambda x: neighbourhood2index(x))
Bijk = Bijk.apply(lambda x: x.apply(lambda y: math.ceil(y)))
Bijk.head()

,nta,0,1,2,3,4,shift
0,79,5,1,4,1,1,1
1,109,4,1,3,1,1,1
2,59,3,1,2,1,1,1
3,110,6,1,4,1,1,1
4,94,4,1,3,1,1,1


In [149]:
Bijk[4].value_counts()

4
1    88
0     1
Name: count, dtype: int64

In [150]:
Skl = pd.read_csv("./aux_data/skl.csv")
newcols = []
for col in Skl.columns[1:]:
    newcols.append(neighbourhood2index(col))
Skl.columns = ["station"]+newcols
Skl["station"] = Skl["station"].apply(lambda x: station2index(x))
Skl.head()

,station,0,1,2,3,4,5,6,7,8,...,185,186,187,188,189,190,191,192,193,194
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [151]:
Pkv = pd.read_csv("./aux_data/pkv.csv", index_col=0)
boroughs = list(Pkv["index"].unique())

def index2boro(l):
    return boroughs[l]

def boro2index(station):
    return boroughs.index(station)

Pkv["index"] = Pkv["index"].apply(lambda x: boro2index(x))
Pkv.columns = np.hstack((np.array("Borough"),pd.Series(list(Pkv.columns[1:])).apply(lambda x: neighbourhood2index(x)).values))
B = len(Pkv["Borough"].unique())
print(B)
Pkv.head()

5


,Borough,0,1,2,3,4,5,6,7,8,...,185,186,187,188,189,190,191,192,193,194
0,0,1,0,1,0,0,1,0,1,1,...,1,0,0,0,0,0,0,0,0,0
1,1,0,1,0,0,1,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
2,2,0,0,0,1,0,0,1,0,0,...,0,0,0,1,1,1,1,0,1,1
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Objective Function Definition

To recap, our objective function has the following aspect:

$$
Minimize \hspace{0.05cm} Z:
\sum_{i}^{I} \sum_{j \in \left\lbrace engine,ladder\right\rbrace} \sum_{k}^{K} \left( B_{i,j,k} - \sum_{l}^{L} x_{i,j,k,l} \right) + \sum_{i}^{I}\sum_{l}^{L}Ch_{l,i}*Yh_{l,i} + \sum_{i}^{I}\sum_{l}^{L}Cs_{l,i}*Ys_{l,i} + \sum_{i}^{I}\sum_{l}^{L}Cr_{l,i}*Yr_{l,i} + \sum_{l1,l2 \in \left\lbrace L\right\rbrace} N_{d->n,l1,l2} + \sum_{l1,l2 \in \left\lbrace L\right\rbrace} N_{n->d,l1,l2}
$$

In [152]:
solver_scip = pywraplp.Solver.CreateSolver("SCIP") # Example, creating a SCIP solver
solver_cbc = pywraplp.Solver.CreateSolver("CBC") # Example, creating a CBC solver
solver_glop = pywraplp.Solver.CreateSolver("GLOP") # Example, creating a SCIP solver

In [153]:
solver = solver_glop

variables = {}

for i in range(I):
  for j in range(J):
    if j not in {1, 3, 4}: # If the vehicle is not of squad, hazardous or rescue type
      for k in range(K):
        for l in range(L):
          # Shifted to integer variable instead of real
          v = solver.IntVar(0, solver.infinity(), "Vehicles of type " + index2vehicle(j) + " assigned to station " + index2station(l) + " to cover the neighbourhood " + index2neighbourhood(k) + " in shift " + index2shift(i))
          variables[(i,j,k,l)] = v
    else:
      for l in range(L):
        v = solver.BoolVar("Vehicles of type " + index2vehicle(j) + " assigned to station " + index2station(l) + " in shift " + index2shift(i))
        variables[(i,j,l)] = v

for l1 in range(L):
  for l2 in range(L):
    v = solver.NumVar(0, solver.infinity(), "Vehicles of type moved from station station " + index2station(l1) + " to station " + index2station(l2) + " in the change of shift (day->night)")
    variables[(0,l1,l2)] = v
    v = solver.NumVar(0, solver.infinity(), "Vehicles of type moved from station station " + index2station(l1) + " to station " + index2station(l2) + " in the change of shift (night->day)")
    variables[(1,l1,l2)] = v

We will add parts to the objective function sequentially

### Add First Sumation

$$
\sum_{i}^{I} \sum_{j \in \left\lbrace engine,ladder\right\rbrace} \sum_{k}^{K} \left( B_{i,j,k} - \sum_{l}^{L} x_{i,j,k,l} \right) 
$$

In [154]:
# 'combinations' dictionary will take account of those combinations of 
# neighbourhoods and shifts where there are no expected needs of vehicles of whatever type
combinations = {}
objective = solver.Objective()
objective.SetMinimization()
for i in range(I):
  for j in range(J):
    if j not in (1,3,4): #Hazmat,squat and rescue units are not included in this summation
      for k in range(K):
        try:
          # The neighbourhood k has needs of vehicles of type j in shift i
          objective.SetOffset(float(Bijk[(Bijk["nta"] == k) & (Bijk["shift"] == i)][j].item()))
          combinations[i,j,k] = "Available"
          for l in range(L):
            v = variables[(i,j,k,l)]
            objective.SetCoefficient(v,-1)
        except:
          combinations[i,j,k] = "Not available"



### Second, Third and Fourth Sumation (Hazmat, Squad and Rescue Allocation Costs)

$$
\sum_{i}^{I}\sum_{l}^{L}Ch_{l,i}*Yh_{l,i} + \sum_{i}^{I}\sum_{l}^{L}Cs_{l,i}*Ys_{l,i} + \sum_{i}^{I}\sum_{l}^{L}Cr_{l,i}*Yr_{l,i}
$$

DE MOMENTO LOS COSTES DE ALLOCATION SON LA MEDIA DE TIEMPO A LOS BARRIOS A LOS QUE SERVIRÍAN. PODRÍAMOS MULTIPLICARLO POR EL NÚMERO MEDIO DE ESTAS UNIDADES ESPERADAS EN ESOS BARRIOS A LOS QUE SERVIRÍAN. (El codigo esta mas menos claculado debajo de la definicion de cost_matriz)

In [155]:
#Squad vehicle
for i in range(I):
    for l in range(L):
        variable = variables[(i,1,l)]
        objective.SetCoefficient(variable,get_costs(l, "squads", i)/total_vehicles["squads"]) #Precoumputed costs are used
 
#Hazardous-Hazmat vehicle
for i in range(I):
    for l in range(L):
        variable = variables[(i,3,l)]
        objective.SetCoefficient(variable,get_costs(l, "ha", i)/total_vehicles["ha"])
        
#Rescue vehicle
for i in range(I):
    for l in range(L):
        variable = variables[(i,4,l)]
        objective.SetCoefficient(variable,get_costs(l, "rescue", i)/total_vehicles["rescue"])       


### Last Sumation

$$
\sum_{l1,l2 \in \left\lbrace L\right\rbrace} N_{d->n,l1,l2} + \sum_{l1,l2 \in \left\lbrace L\right\rbrace} N_{n->d,l1,l2}
$$

In [156]:
for l1 in range(L):
  for l2 in range(L):
    for i in range(I):
      variable = variables[(i,l1,l2)]
      objective.SetCoefficient(variable,1)

## Constraints Definition

### Limits

[limit shift i, vehicle j, neighbouthood k]:$\hspace{0.1cm}\sum_{l}^{L} X_{i,j,k,l} \leq \beta_{i,j,k} \hspace{0.1cm}\forall \hspace{0.1cm}i=1...I,\hspace{0.1cm}j=1...J,\hspace{0.1cm}k=1...K$


In [157]:
limit = []

for i in range(I):
  for j in range(J):
    for k in range(K):
      try:
        bijk = int(Bijk[(Bijk["nta"] == k) & (Bijk["shift"] == i)][j])
        c = solver.Constraint(-solver.infinity(), bijk)
        for l in range(L):
          if (i,j,k,l) in variables:
            v = variables[(i,j,k,l)]
            c.SetCoefficient(v, 1)
        limit.append(c)
      except:
        pass

C:\Users\Hugo\AppData\Local\Temp\ipykernel_15856\3881448656.py:7: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  bijk = int(Bijk[(Bijk["nta"] == k) & (Bijk["shift"] == i)][j])


### Hazmat Vehicle
[hazmat shift i]:$\hspace{0.1cm}\sum_{l}^{L} Yh_{l,i} = 1\hspace{0.1cm}\forall \hspace{0.1cm}i=1...I$

In [158]:
haz = []

for i in range(I):
  c = solver.Constraint(1, 1)
  for l in range(L):
    v = variables[(i,3,l)]
    c.SetCoefficient(v, 1)
  haz.append(c)


### Squads Vehicle

[squads shift i]:$\hspace{0.1cm}\sum_{l}^{L} Ys_{l,i} = 8\hspace{0.1cm}\forall \hspace{0.1cm}i=1...I$

In [159]:
squads = []

for i in range(I):
  c = solver.Constraint(8, 8)
  for l in range(L):
    v = variables[(i,1,l)]
    c.SetCoefficient(v, 1)
  squads.append(c)

### Rescue Vehicle

[squads shift i]:$\hspace{0.1cm}\sum_{l}^{L} Ys_{l,i} = 5\hspace{0.1cm}\forall \hspace{0.1cm}i=1...I$

In [160]:
rescue = []

for i in range(I):
  c = solver.Constraint(5, 5)
  for l in range(L):
    v = variables[(i,4,l)]
    c.SetCoefficient(v, 1)
  rescue.append(c)

### Rest of Vehicles

[vehicle j]:$\hspace{0.1cm}\sum_{i}^{I}\sum_{k}^{K}\sum_{l}^{L} X_{i,j,k,l} \leq\hspace{0.1cm} T_{j}\hspace{0.1cm}\forall \hspace{0.1cm}j=1...J$

In [161]:
vehiclej = []

for j in range(J):
  if j not in {1,3,4}:
    for i in range(I):
      c = solver.Constraint(0, total_vehicles[index2vehicle(j)])
      for l in range(L):
        for k in range(K):
          v = variables[(i,j,k,l)]
          c.SetCoefficient(v, 1)
      vehiclej.append(c)

### Nine-minute arrival

[nine minutes]:$\hspace{0.1cm}\sum_{j}^{J}\sum_{l}^{L} X_{i,j,k,l}*S_{k,l} \geq\hspace{0.1cm} 1\hspace{0.1cm}\forall \hspace{0.1cm}i=1...I \hspace{0.1cm}k=1...K$

In [162]:
nine_minutes = []

for k in range(K):
  for i in range(I):
    c = solver.Constraint(1, solver.infinity())
    for l in range(L):
      skl = s.iloc[l,k]
      for j in range(J):
        if j not in {1, 3, 4}:
          v = variables[(i,j,k,l)]
          c.SetCoefficient(v, skl)
        else:
          v = variables[(i,j,l)]
          c.SetCoefficient(v, skl)

    nine_minutes.append(c)

### Capacities of stations

[capacity]:$\hspace{0.1cm}\sum_{j}^{J}\sum_{k}^{K} X_{i,j,k,l} \leq\hspace{0.1cm} C_{l}\hspace{0.1cm}\forall \hspace{0.1cm}i=1...I \hspace{0.1cm}l=1...L$

In [163]:
capacity = []

for l in range(L):
  cl = Cl[l]
  for i in range(I):
    c = solver.Constraint(-solver.infinity(), cl)
    for j in range(J):
      if j not in {1,3,4}:
        for k in range(K):
          v = variables[(i,j,k,l)]
          c.SetCoefficient(v, 1)
      capacity.append(c)

### Linking the variables of assignment (X) with the displacements (N)

[displacement]:$\hspace{0.1cm}\sum_{j}^{J}\sum_{k}^{K} X_{2,j,k,l} + Yh_{l,2} + Ys_{l,2} + Yr_{l,2} =\hspace{0.1cm}\sum_{j}^{J}\sum_{k}^{K} X_{1,j,k,l} + Yh_{l,1} + Ys_{l,1} + Yr_{l,1} + \sum_{l_{2}}^{L} (N_{d->n,l_{2}, l} - N_{d->n,l,l_{2}}) \hspace{0.1cm}\forall \hspace{0.1cm}l=1...L \hspace{0.1cm}$

In [164]:
displacement2night = []

for l in range(L):
    c = solver.Constraint(-solver.infinity(), 0)
    v = variables[(0,1,l)]
    c.SetCoefficient(v, 1)
    v = variables[(1,1,l)]
    c.SetCoefficient(v, -1)
    v = variables[(1,3,l)]
    c.SetCoefficient(v, -1)
    v = variables[(0,3,l)]
    c.SetCoefficient(v, 1)
    v = variables[(1,4,l)]
    c.SetCoefficient(v, -1)
    v = variables[(0,4,l)]
    c.SetCoefficient(v, 1)
    for l2 in range(L):
        v = variables[(0,l,l2)]
        c.SetCoefficient(v, -1)
        v = variables[(0,l2,l)]
        c.SetCoefficient(v, 1)
    for j in range(J):
      if j not in {1,3,4}:
        for k in range(K):
          v = variables[(0,j,k,l)]
          c.SetCoefficient(v, 1)
          v = variables[(1,j,k,l)]
          c.SetCoefficient(v, -1)
    displacement2night.append(c)

[displacement]:$\hspace{0.1cm}\sum_{j}^{J}\sum_{k}^{K} X_{1,j,k,l} + Yh_{l,1} + Ys_{l,1} + Yr_{l,1} =\hspace{0.1cm}\sum_{j}^{J}\sum_{k}^{K} X_{2,j,k,l} + Yh_{l,2} + Ys_{l,2} + Yr_{l,2} + \sum_{l_{2}}^{L} (N_{n->d,l_{2}, l} - N_{n->d,l,l_{2}}) \hspace{0.1cm}\forall \hspace{0.1cm}l=1...L \hspace{0.1cm}$

In [165]:
displacement2day = []

for l in range(L):
    c = solver.Constraint(-solver.infinity(), 0)
    v = variables[(0,1,l)]
    c.SetCoefficient(v, -1)
    v = variables[(1,1,l)]
    c.SetCoefficient(v, 1)
    v = variables[(1,3,l)]
    c.SetCoefficient(v, 1)
    v = variables[(0,3,l)]
    c.SetCoefficient(v, -1)
    v = variables[(1,4,l)]
    c.SetCoefficient(v, 1)
    v = variables[(0,4,l)]
    c.SetCoefficient(v, -1)
    for l2 in range(L):
        v = variables[(1,l,l2)]
        c.SetCoefficient(v, -1)
        v = variables[(1,l2,l)]
        c.SetCoefficient(v, 1)
    for j in range(J):
      if i not in {1,3,4}:
        for k in range(K):
          v = variables[(0,j,k,l)]
          c.SetCoefficient(v, -1)
          v = variables[(1,j,k,l)]
          c.SetCoefficient(v, 1)
    displacement2day.append(c)

### Ensure a fair distribution

[fair distribution]:$\hspace{0.1cm}\sum_{i}^{I}\sum_{j}^{J}\sum_{k}^{K}\sum_{l}^{L} X_{i,j,k,l}*P_{k,b} \leq\hspace{0.1cm}0.3*\sum_{i}^{I}\sum_{j}^{J}\sum_{k}^{K}\sum_{l}^{L} X_{i,j,k,l} \hspace{0.1cm}\forall \hspace{0.1cm}b=1...B \hspace{0.1cm}$

In [166]:
fair_distribution = []

for b in range(B):
    c = solver.Constraint(-solver.infinity(), 0)
    for k in range(k):
      p = int(Pkv.iloc[b, k+1])
      for j in range(J):
        if j not in {1,3,4}:
          for l in range(L):
            for i in range(I):
              v = variables[(i,j,k,l)]
              c.SetCoefficient(v, p)
              c.SetCoefficient(v, -0.3)
    fair_distribution.append(c)

### Diversity in each station

[not just one type]:$\hspace{0.1cm}\sum_{k}^{K} X_{i,j,k,l} \leq\hspace{0.1cm}\sum_{j}^{J}\sum_{k}^{K} X_{i,j,k,l} - 1 \hspace{0.1cm}\forall \hspace{0.1cm}i=1...I \hspace{0.1cm}j=1...J \hspace{0.1cm}l=1...L$

In [167]:
not_just_one_type = []

for i in range(I):
    for j in range(J):
      for l in range(L):
        c = solver.Constraint(-solver.infinity(), -1)
        for k in range(k):
            v = variables[(i,j,k,l)]
            c.SetCoefficient(v, 1)
            for j2 in range(J):
                if j2 not in {1,3,4}:
                  v = variables[(i,j2,k,l)]
                  c.SetCoefficient(v, -1)
    not_just_one_type.append(c)

In [168]:
result = solver.Solve()

if result == solver.ABNORMAL :
  print("Execution finished by an error")
elif result == solver.FEASIBLE :
  print("In the specified time limit the solver has found a feasible solution")
  for i in range(I):
    for j in range(J):
      if j not in {1, 3, 4}: # If the vehicle is not of squad, hazardous or rescue type
        for k in range(K):
          for l in range(L):
            v = variables[(i,j,k,l)]
            if v.SolutionValue()>0:
              print(v, v.solution_value())
      else:
        for l in range(L):
          v = variables[(i,j,l)]
          if v.SolutionValue()>0:
            print(v, v.solution_value())

  for l1 in range(L):
    for l2 in range(L):
      v = variables[(0,l1,l2)]
      if v.SolutionValue()>0:
        print(v, v.solution_value())
      v = variables[(1,l1,l2)]
      if v.SolutionValue()>0:
        print(v, v.solution_value())
  print("The value for the objective function is", objective.Value())
elif result == solver.INFEASIBLE :
  print("There is no feasible solution for the problem")
elif result == solver.NOT_SOLVED :
  print("In the specified time limit the solver has not found any feasible solution")
elif result == solver.OPTIMAL :
  print("In the specified time limit the solver has found a feasible solution")
  for i in range(I):
    for j in range(J):
      if j not in {1, 3, 4}: # If the vehicle is not of squad, hazardous or rescue type
        for k in range(K):
          for l in range(L):
            v = variables[(i,j,k,l)]
            if v.SolutionValue()>0:
              print(v, v.solution_value())
      else:
        for l in range(L):
          v = variables[(i,j,l)]
          if v.SolutionValue()>0:
            print(v, v.solution_value())

  for l1 in range(L):
    for l2 in range(L):
      v = variables[(0,l1,l2)]
      if v.SolutionValue()>0:
        print(v, v.solution_value())
      v = variables[(1,l1,l2)]
      if v.SolutionValue()>0:
        print(v, v.solution_value())
  print("The optimal value for the objective function is", objective.Value())
elif result == solver.UNBOUNDED :
  print("The solution is unbounded")
else :
  print("Unknown error code")

There is no feasible solution for the problem


There is no feasible solution for the problem